# 1. Create model & train

In [97]:
from sklearn.linear_model import SGDRegressor
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


X_train = pd.read_csv('data/X_train_cleaned.csv')
X_train = X_train.drop(columns=['Id'])
# print(f"type(X_train): {type(X_train)}")
# print(f"X_train.shape: {X_train.shape}")
y_train = pd.read_csv('data/y_train.csv')
# y train should
print(f"type(y_train): {type(y_train)}")
print(f"y_train.shape: {y_train.shape}")


sgd_model = SGDRegressor(
    loss='squared_error',
    alpha=0.01,
    penalty='elasticnet',
    l1_ratio=0.5,   
    shuffle=True, 
    max_iter=1000, 
    verbose=1,
    tol=1e-4
)


sgd_model.fit(X_train, y_train) 

r2_score = sgd_model.score(X_train, y_train)
print(f"R² score: {r2_score:.4f}")

feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': sgd_model.coef_.flatten()
})
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
top_features = feature_importance.sort_values('Abs_Coefficient', ascending=False).head(10)
# print(top_features)

# print(f"Model type {sgd_model.__class__.__name__}")


type(y_train): <class 'pandas.core.frame.DataFrame'>
y_train.shape: (1460, 1)
-- Epoch 1
Norm: 4.61, NNZs: 144, Bias: 11.875839, T: 1460, Avg. loss: 13.530650
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 5.22, NNZs: 126, Bias: 11.662859, T: 2920, Avg. loss: 9.737067
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 5.01, NNZs: 116, Bias: 12.187551, T: 4380, Avg. loss: 11.952929
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 4.59, NNZs: 109, Bias: 11.825526, T: 5840, Avg. loss: 8.258997
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 4.20, NNZs: 78, Bias: 12.124125, T: 7300, Avg. loss: 4.869710
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 3.89, NNZs: 72, Bias: 11.987774, T: 8760, Avg. loss: 2.495698
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 3.78, NNZs: 51, Bias: 12.033065, T: 10220, Avg. loss: 0.582532
Total training time: 0.01 seconds.
-- Epoch 8
Norm: 3.74, NNZs: 52, Bias: 12.022314, T: 11680, Avg. loss: 0.079769
Total training time: 0.01 seconds.
--

/home/mrr/workspaces/kaggle/ml-intro/ml-intro_env/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 2. test model on train output 


In [ ]:
y_pred_log = sgd_model.predict(X_train)
print(f"y_pred_log: {y_pred_log}")
rmse_log = np.sqrt(mean_squared_error(y_train, y_pred_log))
print(f"RMSE (log scale): {rmse_log:.4f}")

# Convert predictions back to original price scale
y_train_actual = np.expm1(y_train)
# y_train_actual = # to series 
# convert y_train_actual to series
y_train_actual = y_train_actual['SalePrice']
print(f"y_train_actual: {y_train_actual}")
y_pred_actual = np.expm1(y_pred_log)

rmse_actual = np.sqrt(mean_squared_error(y_train_actual, y_pred_actual))
print(f"RMSE (actual $): ${rmse_actual:.2f}")

print(f"y_pred_actual: {y_pred_actual}, y_train_actual: {y_train_actual}")
# Mean Absolute Percentage Error (more interpretable)
mape = np.mean(np.abs((y_train_actual - y_pred_actual) / y_train_actual)) * 100
print(f"Mean Absolute Percentage Error: {mape:.2f}%")

import matplotlib.pyplot as plt

# Create scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y_train_actual, y_pred_actual, alpha=0.5)
plt.plot([y_train_actual.min(), y_train_actual.max()], 
         [y_train_actual.min(), y_train_actual.max()], 
         'r--', lw=2)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Predicted vs Actual House Prices')
plt.show()

# Optional: Plot residuals to check for patterns
residuals = y_train_actual - y_pred_actual
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_actual, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# 3. Use model & make predictions

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter


test_df = pd.read_csv('data/X_test_cleaned.csv')
# save the Id column
Id = test_df['Id']
test_df = test_df.drop(columns=['Id'])
predict_prices  = sgd_model.predict(test_df).flatten()
# print(predict_prices)
print(f"predict_prices.shape: {predict_prices.shape}")    

predict_prices_actual = np.expm1(predict_prices)
print(f"Example of prices: \n{predict_prices_actual}")

# Check the values
print("Min price:", predict_prices_actual.min())
print("Max price:", predict_prices_actual.max())
print("Mean price:", predict_prices_actual.mean())


# print 

# Format function to convert to $xxx,xxx format
def currency_formatter(x, pos):
    return f'${x:,.0f}'


plt.figure(figsize=(10, 6))
sns.histplot(predict_prices_actual, kde=True)
plt.title('Distribution of Predicted House Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
# plt.show()
plt.gca().xaxis.set_major_formatter(FuncFormatter(currency_formatter))
plt.tight_layout()
plt.show()





# 4. Create submissional format 

In [84]:
submission_df = pd.DataFrame({
    'Id': Id,
    'SalePrice': predict_prices_actual
})
submission_df.to_csv('data/submission.csv', index=False)